# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

In [ ]:
response.choices[0].message.content

### 封装成一个函数 query_image_description

In [ ]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [ ]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [1]:
from openai import OpenAI
import base64
import requests
import json
import os

client = OpenAI(base_url=os.getenv("OPENAI_BASE_URL"), api_key=os.getenv("OPENAI_API_KEY"))  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post(os.getenv("OPENAI_BASE_URL")+"/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [2]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图表展示了1980年至2020年间美国（蓝色线）、中国（红色线）、日本（紫色线）和德国（绿色线）的国内生产总值（GDP）的比较。数据单位是万亿美元（Trillion USD）。

从图中可以看出，1980年至2020年期间：

1. 美国的GDP从1980年的约3万亿美元稳步上升至2020年的接近21万亿美元，呈现出一条相对平稳且持续增长的趋势。
2. 中国的GDP从1980年的较低水平开始，特别是从1990年代后期开始，经历了显著的增长，到2020年时，GDP接近超过了14万亿美元。
3. 日本的GDP在1980年代和1990年代增长较快，但从1990年代中期开始增速放缓，2000年代初基本保持稳定，在2020年前后略有下降。
4. 德国的GDP增长较为平稳，从1980年的约1万亿美元增长到2020年的接近4万亿美元。

这个图表有效地揭示了这四个国家的经济规模随时间的变化趋势，并突显了中国经济的快速崛起。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [ ]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [ ]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

![](./images/handwriting_1.jpg)

In [ ]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [3]:
content = query_base64_image_description("./images/handwriting_2.jpg")
print(content)

这张图片展示的是一段手写的签名，写着“Marcus Mer”. 这可能是某人的名字或笔名。签名放在一个黄色的背景上。


![](./images/handwriting_2.jpg)